In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import cv2
import glob
import os

from keras.preprocessing import image

Using TensorFlow backend.


In [29]:
DATA_PATH = '/Users/gauravverma/Library/Containers/com.apple.STMExtension.Mail/Data/Documents/Coding_Blocks/MACHINE LEARNING/my-own-projects/Covid_19/covid-chestxray-dataset/'
df = pd.read_csv(DATA_PATH + 'metadata.csv')
df.sample(5)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,needed_supplemental_O2,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 27
73,33,3.0,M,62.0,COVID-19,NaN,NaN,NaN,NaN,NaN,...,"Mar 3, 2020",Italy,images,44C8E3D6-20DA-42E9-B33B-96FA6D6DE12F.jpeg,NaN,https://www.sirm.org/2020/03/03/covid19-caso-2/,NaN,"Remote history changes, not copatologies. Onse...",Credit to UOC Radiology ASST Bergamo Est Direc...,NaN
113,58,7.0,M,36.0,COVID-19,N,Y,Y,NaN,NaN,...,January 13,"Hospital of Wuhan University, Wuhan, China",images,all14238-fig-0002-m-e.jpg,10.1111/all.14238,https://onlinelibrary.wiley.com/doi/full/10.11...,NaN,Chest Xay and CT scan images of a 36ld man wit...,NaN,NaN
155,83,10.0,M,40.0,COVID-19,NaN,NaN,NaN,NaN,NaN,...,2020,Italy,images,5A78BCA9-5B7A-440D-8A4E-AE7710EA6EAD.jpeg,NaN,https://www.sirm.org/2020/03/21/covid-19-caso-57/,NaN,"A 40-year-old male patient, he entered the DEA...","Credit to Priotto Roberto, Negri Alberto, Dema...",NaN
132,68,15.0,NaN,NaN,COVID-19,Y,NaN,NaN,NaN,NaN,...,"Feb 17, 2020",Taiwan,images,1-s2.0-S1684118220300682-main.pdf-003-b1.png,10.1016/j.jmii.2020.03.008,https://www.sciencedirect.com/science/article/...,CC BY-NC-ND,Left lower lung interstitial infiltrates were ...,NaN,NaN
53,22,10.0,M,70.0,COVID-19,NaN,NaN,NaN,NaN,NaN,...,2020,"Riccione, Italy",volumes,radiopaedia_org_covid-19-pneumonia-7_85703_0-d...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,There are large areas of ground glass opacitie...,"Case courtesy of Dr Domenico Nicoletti, Radiop...",NaN


## Gathering Covid-19 X-Ray Scans and dividing in training and validation dataset

In [26]:
DATA_PATH = '/Users/gauravverma/Library/Containers/com.apple.STMExtension.Mail/Data/Documents/Coding_Blocks/MACHINE LEARNING/my-own-projects/Covid_19/covid-chestxray-dataset/'
IMAGE_PATH = DATA_PATH + 'images/'
TARGET_TRAIN_PATH = './Dataset3/Train/Covid-19/'
TARGET_VAL_PATH = './Dataset3/Val/Covid-19/'

try:
    os.mkdir('./Dataset3')
    os.mkdir('./Dataset3/Train')
    os.mkdir('./Dataset3/Train/Covid-19')
    os.mkdir('./Dataset3/Val')
    os.mkdir('./Dataset3/Val/Covid-19')
             
except:
#     print('error')
    pass

count = 0

for i, row in df.iterrows():
    if row['finding'].startswith('COVID-19') and row['view'] == 'PA':
        train_image = IMAGE_PATH + row['filename']
        if np.random.rand() > 0.2:
            target_file = TARGET_TRAIN_PATH + row['filename']
        else:
            target_file = TARGET_VAL_PATH + row['filename']
        shutil.copy2(train_image, target_file)
        count+=1
        print(f'Copying: {count}')
count

Copying: 1
Copying: 2
Copying: 3
Copying: 4
Copying: 5
Copying: 6
Copying: 7
Copying: 8
Copying: 9
Copying: 10
Copying: 11
Copying: 12
Copying: 13
Copying: 14
Copying: 15
Copying: 16
Copying: 17
Copying: 18
Copying: 19
Copying: 20
Copying: 21
Copying: 22
Copying: 23
Copying: 24
Copying: 25
Copying: 26
Copying: 27
Copying: 28
Copying: 29
Copying: 30
Copying: 31
Copying: 32
Copying: 33
Copying: 34
Copying: 35
Copying: 36
Copying: 37
Copying: 38
Copying: 39
Copying: 40
Copying: 41
Copying: 42
Copying: 43
Copying: 44
Copying: 45
Copying: 46
Copying: 47
Copying: 48
Copying: 49
Copying: 50
Copying: 51
Copying: 52
Copying: 53
Copying: 54
Copying: 55
Copying: 56
Copying: 57
Copying: 58
Copying: 59
Copying: 60
Copying: 61
Copying: 62
Copying: 63
Copying: 64
Copying: 65
Copying: 66
Copying: 67
Copying: 68
Copying: 69
Copying: 70
Copying: 71
Copying: 72
Copying: 73
Copying: 74
Copying: 75
Copying: 76
Copying: 77
Copying: 78
Copying: 79
Copying: 80
Copying: 81
Copying: 82
Copying: 83
Copying: 84
C

153

## Gathering Normal X-Ray Scans and dividing in training and validation dataset

In [28]:
IMAGE_PATH = '/Users/gauravverma/Library/Containers/com.apple.STMExtension.Mail/Data/Documents/Coding_Blocks/MACHINE LEARNING/my-own-projects/Covid_19/chest_xray/chest_xray/train/NORMAL/'
TARGET_TRAIN_PATH = './Dataset3/Train/Normal/'
TARGET_TEST_PATH = './Dataset3/Val/Normal/'

try:
    os.mkdir('./Dataset3')
    os.mkdir('./Dataset3/Train')
    os.mkdir('./Dataset3/Train/Normal')
    os.mkdir('./Dataset3/Val')
    os.mkdir('./Dataset3/Val/Normal')
except:
    pass

train_images = glob.glob(IMAGE_PATH + '*')

np.random.shuffle(train_images)

count = 0

for i in range(153):
    img_name = train_images[i].split('/')[-1]
    if np.random.rand() > 0.2:
        target_file = TARGET_TRAIN_PATH + img_name
    else:
        target_file = TARGET_VAL_PATH + img_name
    shutil.copy2(train_images[i], target_file)
    count+=1
    print(f'Copying: {count}')

Copying: 1
Copying: 2
Copying: 3
Copying: 4
Copying: 5
Copying: 6
Copying: 7
Copying: 8
Copying: 9
Copying: 10
Copying: 11
Copying: 12
Copying: 13
Copying: 14
Copying: 15
Copying: 16
Copying: 17
Copying: 18
Copying: 19
Copying: 20
Copying: 21
Copying: 22
Copying: 23
Copying: 24
Copying: 25
Copying: 26
Copying: 27
Copying: 28
Copying: 29
Copying: 30
Copying: 31
Copying: 32
Copying: 33
Copying: 34
Copying: 35
Copying: 36
Copying: 37
Copying: 38
Copying: 39
Copying: 40
Copying: 41
Copying: 42
Copying: 43
Copying: 44
Copying: 45
Copying: 46
Copying: 47
Copying: 48
Copying: 49
Copying: 50
Copying: 51
Copying: 52
Copying: 53
Copying: 54
Copying: 55
Copying: 56
Copying: 57
Copying: 58
Copying: 59
Copying: 60
Copying: 61
Copying: 62
Copying: 63
Copying: 64
Copying: 65
Copying: 66
Copying: 67
Copying: 68
Copying: 69
Copying: 70
Copying: 71
Copying: 72
Copying: 73
Copying: 74
Copying: 75
Copying: 76
Copying: 77
Copying: 78
Copying: 79
Copying: 80
Copying: 81
Copying: 82
Copying: 83
Copying: 84
C